# Read and write CPACS using TiXI

[TiXI](https://github.com/DLR-SC/tixi):
 - Binary Downloads:  https://github.com/DLR-SC/tixi/wiki/Downloads
 - API Documentation: http://tixi.sourceforge.net/Doc/index.html
 - Issue Tracker:     https://github.com/DLR-SC/tixi/issues
 - Wiki:              https://github.com/DLR-SC/tixi/wiki

In [9]:
import os
from tixi3 import tixi3wrapper
tixi_h = tixi3wrapper.Tixi3()

In [2]:
help(tixi_h)

Help on Tixi3 in module tixi3.tixi3wrapper object:

class Tixi3(builtins.object)
 |  Methods defined here:
 |  
 |  __del__(self)
 |  
 |  __init__(self)
 |      Initialize self.  See help(type(self)) for accurate signature.
 |  
 |  addBooleanElement(self, parentPath, elementName, boolean)
 |  
 |  addBooleanElementNS(self, parentPath, qualifiedName, namespaceURI, boolean)
 |  
 |  addCpacsHeader(self, name, creator, version, description, cpacsVersion)
 |  
 |  addDoubleAttribute(self, elementPath, attributeName, number, format)
 |  
 |  addDoubleElement(self, parentPath, elementName, number, format)
 |  
 |  addDoubleElementNS(self, parentPath, qualifiedName, namespaceURI, number, format)
 |  
 |  addDoubleListWithAttributes(self, parentPath, listName, childName, childAttributeName, values, format, attributes, nValues)
 |  
 |  addExternalLink(self, parentPath, url, fileFormat)
 |  
 |  addFloatVector(self, parentPath, elementName, vector, numElements, format)
 |  
 |  addHeader(self

Open a CPACS file with tixi:

In [3]:
tixi_h.open('D150_v30.xml')

Get airport information:

In [4]:
# airport_xpath = '/cpacs/airports/airport[1]'
airport_xpath = tixi_h.uIDGetXPath('FRA')

airport_name = tixi_h.getTextElement(os.path.join(airport_xpath,'name'))
positionNorth = tixi_h.getDoubleElement(os.path.join(airport_xpath,'positionNorth'))
positionEast = tixi_h.getDoubleElement(os.path.join(airport_xpath,'positionEast'))

print('Name: %s\nPosition North: %.3f\nPosition East: %.3f'
        %(airport_name, positionNorth, positionEast))

Name: Frankfurt Main International Airport, Germany
Position North: 50.034
Position East: 8.571


Some information would be very long if stored as xml-sequence. Thus, we introduced vectors:

In [5]:
root_xpath = os.path.join(tixi_h.uIDGetXPath('FRAtoEWRID'),'flightPath')

vector_size = tixi_h.getVectorSize(os.path.join(root_xpath,'waypoints'))

waypoints = tixi_h.getTextElement(os.path.join(root_xpath,'waypoints')).split(';')
latitude = tixi_h.getFloatVector(os.path.join(root_xpath,'latitude'),vector_size)
longitude = tixi_h.getFloatVector(os.path.join(root_xpath,'longitude'),vector_size)

for i, wp in enumerate(waypoints): print('%10s %.3f %.3f'%(wp, latitude[i], longitude[i]))

      EDDF 50.033 8.570
     ESUPI 50.058 8.121
     MASIR 50.255 7.738
     RAVKI 50.362 7.665
     DITAM 50.558 7.530
     ABAXA 50.765 7.386
     BIBOS 50.949 7.209
     DISMO 51.409 6.752
     NAPSI 51.855 6.021
       SPY 52.540 4.854
     BERGI 52.749 4.359
     MIMVA 53.101 3.304
     LEGRO 53.326 1.513
     NALAX 53.483 0.402
     REMSI 53.960 -3.826
     BESOP 54.547 -5.446
       BEL 54.661 -6.230
     NIBOG 55.000 -10.000
     ODLUM 55.000 -12.000
     RESNO 55.000 -15.000
     OYSTR 53.867 -54.967
     STEAM 52.998 -57.342
       ALB 42.747 -73.803
     ACOVE 42.235 -74.032
     TALCO 42.063 -74.108
     WEETS 41.858 -74.198
     HELON 41.667 -74.280
     FLOSI 41.544 -74.334
     CRANK 41.417 -74.389
     SHAFF 41.290 -74.443
       SAX 41.068 -74.538
     PATRN 40.941 -74.151
       TEB 40.849 -74.062
     GIMEE 40.813 -74.088
     AYRON 40.765 -74.121
      KEWR 40.693 -74.169


Let's add information about Braunschweig airport:

In [6]:
apts_xpath = '/cpacs/airports'
idx = tixi_h.getNamedChildrenCount(apts_xpath,'airport') + 1

tixi_h.createElementAtIndex(apts_xpath,'airport',idx)

apt_xpath = os.path.join(apts_xpath,'airport[%i]'%idx)

tixi_h.addTextAttribute(apt_xpath,'uID','BWE')
tixi_h.addTextElement(apt_xpath, 'name', 'Braunschweig, Germany')
tixi_h.addDoubleElement(apt_xpath, 'positionNorth', 52.319, '%.3f')
tixi_h.addDoubleElement(apt_xpath, 'positionEast', 10.556, '%.3f')
tixi_h.addIntegerElement(apt_xpath, 'elevation', 84, '%i')

Save the modified CPACS instance as a new file:

In [7]:
tixi_h.save('D150_v30_modified.xml')
tixi_h.close()